DQN ROBUSTNESS UNDER STOCHASTICITY

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback
from pathlib import Path
from dataclasses import dataclass, asdict
from typing import Optional, Dict, Tuple, List
import imageio.v2 as imageio
import warnings

warnings.filterwarnings('ignore')

In [15]:
SEEDS = [0, 42, 1729, 6174, 196]
FAILURE_PROBS = [0.00, 0.05, 0.10, 0.15, 0.20, 0.25]
TRAINING_TIMESTEPS = 700_000
N_EVAL_EPISODES = 100

# FIX #7: Target update interval set to 10,000
# FIX #8: Exploration fraction = 0.2, final eps = 0.05
DQN_CONFIG = {
    'learning_rate': 1e-3,
    'buffer_size': 100_000,
    'learning_starts': 10_000,
    'batch_size': 64,
    'gamma': 0.99,
    'train_freq': 4,
    'gradient_steps': 1,
    'target_update_interval': 10_000,  # FIX #7
    'exploration_fraction': 0.2,        # FIX #8
    'exploration_initial_eps': 1.0,
    'exploration_final_eps': 0.05,      # FIX #8
    'verbose': 0,
}

sns.set_style("whitegrid")
plt.rcParams.update({'figure.dpi': 150, 'font.size': 11})

print("✓ Configuration loaded")

✓ Configuration loaded


### ENVIRONMENT WRAPPERS

In [16]:
class StochasticLunarLanderWrapper(gym.Wrapper):
    """
    FIX #2: Single wrapper that implements ONE stochastic family per episode.
    At reset, samples Bernoulli(p):
    - If false: no failures
    - If true: exactly ONE family is activated
    
    FIX #4: Tracks failure_occurred and failure_step globally.
    """
    
    FAMILIES = ['sensor', 'actuator', 'dynamics', 'catastrophic']
    
    def __init__(self, env, failure_prob=0.0, seed=None):
        super().__init__(env)
        self.failure_prob = failure_prob
        
        # FIX #3: Single RNG initialization (no env.env.env crawling)
        self.rng = np.random.RandomState(seed)
        
        # FIX #2: Active family selection
        self.active_family = None
        
        # FIX #4: Shared failure tracking
        self.failure_occurred = False
        self.failure_step = None
        self.current_step = 0
        
        # Family-specific state
        self._reset_family_states()
        
        # Last observation for sensor dropout
        self.last_obs = None
    
    def _reset_family_states(self):
        """Reset all family-specific failure states"""
        # Sensor failures
        self.altimeter_drift_enabled = False
        self.altimeter_bias = 0.0
        
        # Actuator failures
        self.engine_degradation_active = False
        self.thrust_multiplier = 1.0
        self.thruster_asymmetry_active = False
        self.left_mult = 1.0
        self.right_mult = 1.0
        self.control_lag_active = False
        self.action_buffer = []
        
        # Dynamics failures
        self.fuel_leak_active = False
        self.leak_rate = 0.0
        self.gravity_variation_active = False
        self.gravity_mult = 1.0
        self.current_mass = 1.0
        
        # Catastrophic failures
        self.engine_cutoff_active = False
        self.engine_cutoff_step = -1
        self.landing_gear_failure = False
    
    def reset(self, **kwargs):
        # FIX #3: Use seed parameter properly
        obs, info = self.env.reset(**kwargs)
        
        # FIX #2: Single Bernoulli sampling
        self._reset_family_states()
        self.current_step = 0
        self.failure_occurred = False
        self.failure_step = None
        self.last_obs = obs.copy()
        
        if self.rng.random() < self.failure_prob:
            # Failure WILL occur - select ONE family
            self.active_family = self.rng.choice(self.FAMILIES)
            self._initialize_active_family()
        else:
            # NO failures this episode
            self.active_family = None
        
        info['active_family'] = self.active_family
        return obs, info
    
    def _initialize_active_family(self):
        """Initialize the selected stochastic family"""
        if self.active_family == 'sensor':
            self.altimeter_drift_enabled = True
            
        elif self.active_family == 'actuator':
            # Randomly select one actuator failure mode
            mode = self.rng.choice(['degradation', 'asymmetry'])
            if mode == 'degradation':
                self.engine_degradation_active = True
                self.thrust_multiplier = self.rng.uniform(0.3, 0.6)
            else:
                self.thruster_asymmetry_active = True
                weakness = self.rng.uniform(0.4, 0.8)
                if self.rng.random() < 0.5:
                    self.left_mult, self.right_mult = weakness, 1.0
                else:
                    self.left_mult, self.right_mult = 1.0, weakness
                    
        elif self.active_family == 'dynamics':
            # Randomly select one dynamics failure mode
            mode = self.rng.choice(['fuel_leak', 'gravity'])
            if mode == 'fuel_leak':
                self.fuel_leak_active = True
                self.leak_rate = self.rng.uniform(0.001, 0.003)
            else:
                self.gravity_variation_active = True
                self.gravity_mult = self.rng.uniform(0.9, 1.1)
                
        elif self.active_family == 'catastrophic':
            # Randomly select one catastrophic failure mode
            mode = self.rng.choice(['cutoff', 'landing_gear'])
            if mode == 'cutoff':
                self.engine_cutoff_active = True
                self.engine_cutoff_step = self.rng.randint(100, 600)
            else:
                self.landing_gear_failure = True
    
    def observation(self, obs):
        """Apply sensor failures if active"""
        if self.active_family != 'sensor':
            return obs
        
        obs = obs.copy()
        
        # FIX #4: Mark first failure
        if not self.failure_occurred:
            self.failure_occurred = True
            self.failure_step = self.current_step
        
        # Altimeter drift (episodic + stepwise)
        if self.altimeter_drift_enabled:
            self.altimeter_bias += self.rng.uniform(-0.02, 0.02)
            obs[1] += self.altimeter_bias
        
        # Gyroscope malfunction (stepwise)
        if self.rng.random() < 0.3:
            obs[4] *= self.rng.choice([0.5, 1.5, -1.0])
        
        # Velocity sensor error (stepwise)
        if self.rng.random() < 0.3:
            obs[2] += self.rng.uniform(-1.0, 1.0)
        
        # Sensor dropout (stepwise)
        if self.rng.random() < 0.3:
            dropout_idx = self.rng.choice([0, 1, 4])
            if self.rng.random() < 0.5:
                obs[dropout_idx] = 0.0
            elif self.last_obs is not None:
                obs[dropout_idx] = self.last_obs[dropout_idx]
        
        self.last_obs = obs.copy()
        return obs
    
    def step(self, action):
        self.current_step += 1
        
        # Apply actuator failures
        if self.active_family == 'actuator':
            action = self._apply_actuator_failures(action)
        
        # Apply catastrophic failures
        if self.active_family == 'catastrophic':
            action = self._apply_catastrophic_failures(action)
        
        obs, reward, terminated, truncated, info = self.env.step(action)
        
        # Apply observation failures
        obs = self.observation(obs)
        
        # Apply dynamics failures
        if self.active_family == 'dynamics':
            self._apply_dynamics_failures(action)
        
        # Apply physics modifications
        if self.active_family == 'actuator':
            self._apply_actuator_physics()
        
        # Handle catastrophic landing gear failure
        if self.active_family == 'catastrophic' and self.landing_gear_failure:
            if terminated and reward > 0:
                reward = -100
                info['landing_gear_crash'] = True
        
        return obs, reward, terminated, truncated, info
    
    def _apply_actuator_failures(self, action):
        """Apply actuator failures to action"""
        # FIX #4: Mark first failure
        if not self.failure_occurred:
            self.failure_occurred = True
            self.failure_step = self.current_step
        
        # Control lag
        if not self.control_lag_active and self.rng.random() < 0.1:
            self.control_lag_active = True
            self.action_buffer = []
        
        if self.control_lag_active:
            self.action_buffer.append(action)
            if len(self.action_buffer) > self.rng.randint(1, 4):
                action = self.action_buffer.pop(0)
            else:
                action = 0
        
        return action
    
    def _apply_actuator_physics(self):
        """Apply actuator physics modifications"""
        try:
            lander = self.env.unwrapped.lander
            if lander is None:
                return
            
            if self.engine_degradation_active:
                vel = lander.linearVelocity
                if vel.y > 0:
                    lander.linearVelocity = (
                        vel.x, 
                        vel.y * (1.0 - (1.0 - self.thrust_multiplier) * 0.15)
                    )
            
            if self.thruster_asymmetry_active:
                ang_vel = lander.angularVelocity
                if self.left_mult < 1.0 and ang_vel > 0:
                    lander.angularVelocity = ang_vel * self.left_mult
                elif self.right_mult < 1.0 and ang_vel < 0:
                    lander.angularVelocity = ang_vel * self.right_mult
        except:
            pass
    
    def _apply_dynamics_failures(self, action):
        """Apply dynamics failures"""
        # FIX #4: Mark first failure
        if not self.failure_occurred:
            self.failure_occurred = True
            self.failure_step = self.current_step
        
        try:
            lander = self.env.unwrapped.lander
            if lander is None:
                return
            
            # Mass depletion
            if action == 2:
                self.current_mass -= 0.001
                self.current_mass = max(0.5, self.current_mass)
            
            # Fuel leak
            if self.fuel_leak_active:
                self.current_mass -= self.leak_rate
                self.current_mass = max(0.5, self.current_mass)
                if action == 2:
                    vel = lander.linearVelocity
                    lander.linearVelocity = (vel.x, vel.y * 0.95)
            
            # Gravity variation
            if self.gravity_variation_active:
                vel = lander.linearVelocity
                correction = (self.gravity_mult - 1.0) * 0.05
                lander.linearVelocity = (vel.x, vel.y - correction)
            
            # Wind turbulence
            if self.rng.random() < 0.2:
                wind = self.rng.uniform(-2.0, 2.0)
                vel = lander.linearVelocity
                lander.linearVelocity = (vel.x + wind * 0.1, vel.y)
        except:
            pass
    
    def _apply_catastrophic_failures(self, action):
        """Apply catastrophic failures"""
        # FIX #4: Mark first failure at cutoff point
        if self.engine_cutoff_active and self.current_step >= self.engine_cutoff_step:
            if not self.failure_occurred:
                self.failure_occurred = True
                self.failure_step = self.current_step
            if action == 2:
                action = 0
        
        return action


def make_stochastic_lunar_lander(failure_prob, seed=None):
    """
    FIX #1: Create single environment instance (NO vectorization)
    FIX #3: Proper seed handling
    """
    env = gym.make('LunarLander-v2')
    env = StochasticLunarLanderWrapper(env, failure_prob, seed)
    
    # FIX #3: Use reset with seed parameter (no manual crawling)
    if seed is not None:
        env.reset(seed=seed)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
    
    return env

print("✓ Environment wrappers loaded")

✓ Environment wrappers loaded


### Training

In [17]:
class TrainingProgressCallback(BaseCallback):
    def __init__(self, check_freq=10000, verbose=0):
        super().__init__(verbose)
        self.check_freq = check_freq
    
    def _on_step(self):
        return True


def train_dqn_agent(failure_prob, seed, save_dir, verbose=1):
    """
    FIX #1: Train on SINGLE environment (no vectorization)
    """
    if verbose:
        print(f"\nTraining DQN | p={failure_prob:.2f} | seed={seed}")
    
    # FIX #1: Single environment wrapped in Monitor
    env = Monitor(make_stochastic_lunar_lander(failure_prob, seed))
    model = DQN('MlpPolicy', env, seed=seed, **DQN_CONFIG)
    
    callback = TrainingProgressCallback(10000, verbose)
    model.learn(TRAINING_TIMESTEPS, callback=callback, progress_bar=(verbose > 0))
    
    save_dir.mkdir(parents=True, exist_ok=True)
    model_path = save_dir / f"dqn_p{int(failure_prob*100):02d}_seed{seed}.zip"
    model.save(str(model_path))
    
    if verbose:
        print(f"✓ Saved: {model_path}")
    
    env.close()
    return model


def train_experiment1(output_dir=Path("models/exp1"), verbose=1):
    print("\n" + "="*70)
    print("EXPERIMENT 1: Training 30 agents (6 levels × 5 seeds)")
    print("="*70)
    
    models = {}
    for p in FAILURE_PROBS:
        for seed in SEEDS:
            models[(p, seed)] = train_dqn_agent(p, seed, output_dir, verbose)
    
    print("\n✓ Experiment 1 training complete")
    return models


def train_experiment2(output_dir=Path("models/exp2"), verbose=1):
    print("\n" + "="*70)
    print("EXPERIMENT 2: Training 5 agents (p=0.0 only)")
    print("="*70)
    
    models = {}
    for seed in SEEDS:
        models[(0.0, seed)] = train_dqn_agent(0.0, seed, output_dir, verbose)
    
    print("\n✓ Experiment 2 training complete")
    return models

In [18]:
def load_all_models_exp1(model_dir=Path("models/exp1")):
    models = {}
    for p in FAILURE_PROBS:
        for seed in SEEDS:
            path = model_dir / f"dqn_p{int(p*100):02d}_seed{seed}.zip"
            if path.exists():
                models[(p, seed)] = DQN.load(str(path))
    print(f"✓ Loaded {len(models)}/30 models")
    return models


def load_all_models_exp2(model_dir=Path("models/exp2")):
    models = {}
    for seed in SEEDS:
        path = model_dir / f"dqn_p00_seed{seed}.zip"
        if path.exists():
            models[(0.0, seed)] = DQN.load(str(path))
    print(f"✓ Loaded {len(models)}/5 models")
    return models

print("✓ Training module loaded")

✓ Training module loaded


### Evaluation

Separate metrics for each experiment

In [19]:
@dataclass
class Exp1EpisodeMetrics:
    """FIX #6: Experiment 1 metrics only"""
    episode_id: int
    seed: int
    failure_prob: float
    total_return: float
    episode_length: int
    success: bool
    crash: bool


@dataclass
class Exp2EpisodeMetrics:
    """FIX #6: Experiment 2 metrics (includes failure tracking)"""
    episode_id: int
    seed: int
    failure_prob: float
    total_return: float
    episode_length: int
    success: bool
    crash: bool
    failure_occurred: bool
    time_to_failure: int  # FIX #5: Steps until first failure
    pre_fault_return: float
    post_fault_return: float


@dataclass
class Exp1AggregatedMetrics:
    """FIX #6: Experiment 1 aggregated metrics"""
    failure_prob: float
    seed: int
    n_episodes: int
    mean_return: float
    std_return: float
    success_rate: float
    crash_rate: float


@dataclass
class Exp2AggregatedMetrics:
    """FIX #6: Experiment 2 aggregated metrics"""
    failure_prob: float
    seed: int
    n_episodes: int
    mean_return: float
    std_return: float
    success_rate: float
    crash_rate: float
    mean_time_to_failure: float  # FIX #5
    mean_pre_fault_return: float
    mean_post_fault_return: float


def evaluate_single_episode_exp1(model, env, ep_id, seed, failure_prob):
    """
    FIX #6: Experiment 1 evaluation (no failure tracking)
    """
    obs, info = env.reset()
    
    ret, length = 0.0, 0
    done = False
    
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)
        ret += reward
        length += 1
        done = terminated or truncated
    
    metrics = Exp1EpisodeMetrics(
        episode_id=ep_id,
        seed=seed,
        failure_prob=failure_prob,
        total_return=ret,
        episode_length=length,
        success=(ret >= 200),
        crash=(ret < -100)
    )
    
    return metrics


def evaluate_single_episode_exp2(model, env, ep_id, seed, failure_prob, record=False):
    """
    FIX #4: Track failure_occurred and failure_step from environment
    FIX #5: Compute time_to_failure correctly
    FIX #6: Experiment 2 evaluation (full metrics)
    """
    obs, info = env.reset()
    
    ret, length = 0.0, 0
    pre_ret, post_ret = 0.0, 0.0
    frames = [] if record else None
    done = False
    
    while not done:
        if record:
            frame = env.render()
            if frame is not None:
                frames.append(frame)
        
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)
        
        ret += reward
        length += 1
        done = terminated or truncated
        
        # FIX #4: Use environment's failure tracking
        if env.failure_occurred and env.failure_step is not None:
            if length <= env.failure_step:
                pre_ret += reward
            else:
                post_ret += reward
        else:
            pre_ret += reward
    
    # FIX #5: Time-to-failure = steps until first failure (or episode length if none)
    if env.failure_occurred and env.failure_step is not None:
        time_to_failure = env.failure_step
    else:
        time_to_failure = length
    
    metrics = Exp2EpisodeMetrics(
        episode_id=ep_id,
        seed=seed,
        failure_prob=failure_prob,
        total_return=ret,
        episode_length=length,
        success=(ret >= 200),
        crash=(ret < -100),
        failure_occurred=env.failure_occurred,
        time_to_failure=time_to_failure,
        pre_fault_return=pre_ret,
        post_fault_return=post_ret
    )
    
    return metrics, frames


def evaluate_agent_exp1(model, failure_prob, seed, n_episodes=100):
    """FIX #6: Experiment 1 evaluation"""
    env = make_stochastic_lunar_lander(failure_prob, seed)
    
    metrics_list = []
    for ep in range(n_episodes):
        metrics = evaluate_single_episode_exp1(model, env, ep, seed, failure_prob)
        metrics_list.append(metrics)
    
    env.close()
    return metrics_list


def evaluate_agent_exp2(model, failure_prob, seed, n_episodes=100, record_last=0):
    """FIX #6: Experiment 2 evaluation (with failure tracking)"""
    env = make_stochastic_lunar_lander(failure_prob, seed)
    
    if record_last > 0:
        env = gym.make('LunarLander-v2', render_mode='rgb_array')
        env = StochasticLunarLanderWrapper(env, failure_prob, seed)
        env.reset(seed=seed)
    
    metrics_list = []
    all_frames = []
    
    for ep in range(n_episodes):
        should_record = (ep >= n_episodes - record_last) and record_last > 0
        metrics, frames = evaluate_single_episode_exp2(
            model, env, ep, seed, failure_prob, should_record
        )
        metrics_list.append(metrics)
        if frames:
            all_frames.extend(frames)
    
    env.close()
    return metrics_list, (all_frames if all_frames else None)


def aggregate_metrics_exp1(metrics_list):
    """FIX #6: Experiment 1 aggregation"""
    returns = [m.total_return for m in metrics_list]
    successes = [m.success for m in metrics_list]
    crashes = [m.crash for m in metrics_list]
    
    return Exp1AggregatedMetrics(
        failure_prob=metrics_list[0].failure_prob,
        seed=metrics_list[0].seed,
        n_episodes=len(metrics_list),
        mean_return=np.mean(returns),
        std_return=np.std(returns),
        success_rate=np.mean(successes),
        crash_rate=np.mean(crashes)
    )


def aggregate_metrics_exp2(metrics_list):
    """FIX #6: Experiment 2 aggregation (includes failure metrics)"""
    returns = [m.total_return for m in metrics_list]
    successes = [m.success for m in metrics_list]
    crashes = [m.crash for m in metrics_list]
    ttf = [m.time_to_failure for m in metrics_list]
    pre_rets = [m.pre_fault_return for m in metrics_list]
    post_rets = [m.post_fault_return for m in metrics_list]
    
    return Exp2AggregatedMetrics(
        failure_prob=metrics_list[0].failure_prob,
        seed=metrics_list[0].seed,
        n_episodes=len(metrics_list),
        mean_return=np.mean(returns),
        std_return=np.std(returns),
        success_rate=np.mean(successes),
        crash_rate=np.mean(crashes),
        mean_time_to_failure=np.mean(ttf),
        mean_pre_fault_return=np.mean(pre_rets),
        mean_post_fault_return=np.mean(post_rets)
    )


def evaluate_experiment1(models, n_episodes=100, output_dir=Path("results/exp1")):
    """FIX #6: Experiment 1 evaluation pipeline"""
    print("\n" + "="*70)
    print("EXPERIMENT 1: Cross-evaluation")
    print("="*70)
    
    results = []
    for (train_p, train_seed), model in models.items():
        print(f"\nModel trained at p={train_p:.2f}, seed={train_seed}")
        for test_p in FAILURE_PROBS:
            print(f"  Testing at p={test_p:.2f}...", end=" ")
            metrics_list = evaluate_agent_exp1(model, test_p, train_seed, n_episodes)
            agg = aggregate_metrics_exp1(metrics_list)
            
            result = {
                'train_p': train_p,
                'test_p': test_p,
                'seed': train_seed,
                **asdict(agg)
            }
            results.append(result)
            print(f"✓ (return={agg.mean_return:.1f})")
    
    df = pd.DataFrame(results)
    output_dir.mkdir(parents=True, exist_ok=True)
    csv_path = output_dir / "exp1_results.csv"
    df.to_csv(csv_path, index=False)
    print(f"\n✓ Saved: {csv_path}")
    return df


def evaluate_experiment2(models, n_episodes=100, record_gifs=True, output_dir=Path("results/exp2")):
    """
    FIX #6: Experiment 2 evaluation pipeline (with failure tracking)
    FIX #9: GIF speed = 2× real-time (fps=100)
    """
    print("\n" + "="*70)
    print("EXPERIMENT 2: Zero-shot evaluation")
    print("="*70)
    
    results = []
    for (_, seed), model in models.items():
        print(f"\nModel seed={seed}")
        for test_p in FAILURE_PROBS:
            print(f"  Testing at p={test_p:.2f}...", end=" ")
            record_n = 10 if record_gifs else 0
            metrics_list, frames = evaluate_agent_exp2(
                model, test_p, seed, n_episodes, record_n
            )
            agg = aggregate_metrics_exp2(metrics_list)
            
            result = {'train_p': 0.0, 'test_p': test_p, 'seed': seed, **asdict(agg)}
            results.append(result)
            
            if frames:
                gif_dir = output_dir / "gifs"
                gif_dir.mkdir(parents=True, exist_ok=True)
                gif_path = gif_dir / f"exp2_p{int(test_p*100):02d}_seed{seed}.gif"
                # FIX #9: 2× real-time playback via fps=100
                imageio.mimsave(str(gif_path), frames, fps=100, loop=0)
                print(f"✓ GIF saved (2× speed)", end=" ")
            
            print(f"(return={agg.mean_return:.1f})")
    
    df = pd.DataFrame(results)
    output_dir.mkdir(parents=True, exist_ok=True)
    csv_path = output_dir / "exp2_results.csv"
    df.to_csv(csv_path, index=False)
    print(f"\n✓ Saved: {csv_path}")
    return df

print("✓ Evaluation module loaded")

✓ Evaluation module loaded


### Visualization

In [20]:
def plot_exp1_heatmap(df, metric, output_dir):
    pivot = df.groupby(['train_p', 'test_p'])[metric].mean().unstack()
    if 'rate' in metric:
        pivot *= 100
    
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(pivot, annot=True, fmt='.1f', cmap='RdYlGn', center=50 if 'rate' in metric else 150,
                ax=ax, linewidths=1, cbar_kws={'label': metric.replace('_', ' ').title()})
    
    ax.set_xlabel('Test Failure Probability')
    ax.set_ylabel('Training Failure Probability')
    ax.set_title(f'Experiment 1: {metric.replace("_", " ").title()}')
    ax.set_xticklabels([f'{int(p*100)}%' for p in pivot.columns])
    ax.set_yticklabels([f'{int(p*100)}%' for p in pivot.index], rotation=0)
    
    output_dir.mkdir(parents=True, exist_ok=True)
    plt.tight_layout()
    plt.savefig(output_dir / f'exp1_{metric}.png', dpi=300, bbox_inches='tight')
    plt.close()


def plot_exp1_robustness_curves(df, output_dir):
    fig, ax = plt.subplots(figsize=(12, 7))
    
    for train_p in sorted(df['train_p'].unique()):
        subset = df[df['train_p'] == train_p]
        grouped = subset.groupby('test_p')['success_rate'].agg(['mean', 'sem'])
        
        test_ps = [p * 100 for p in grouped.index]
        means = grouped['mean'] * 100
        sems = grouped['sem'] * 100
        
        ax.plot(test_ps, means, marker='o', linewidth=2, markersize=8, 
                label=f'Train p={int(train_p*100)}%')
        ax.fill_between(test_ps, means - 1.96*sems, means + 1.96*sems, alpha=0.2)
    
    ax.axhline(50, color='red', linestyle='--', linewidth=1.5, alpha=0.7, label='50% Threshold')
    ax.set_xlabel('Test Failure Probability (%)')
    ax.set_ylabel('Success Rate (%)')
    ax.set_title('Experiment 1: Robustness Curves')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-5, 105)
    
    plt.tight_layout()
    plt.savefig(output_dir / 'exp1_robustness_curves.png', dpi=300, bbox_inches='tight')
    plt.close()


def plot_exp2_performance(df, metric, output_dir):
    fig, ax = plt.subplots(figsize=(10, 7))
    
    grouped = df.groupby('test_p')[metric].agg(['mean', 'sem'])
    test_ps = [p * 100 for p in grouped.index]
    means = grouped['mean']
    sems = grouped['sem']
    
    if 'rate' in metric:
        means *= 100
        sems *= 100
    
    ax.plot(test_ps, means, marker='o', linewidth=3, markersize=10, color='steelblue')
    ax.fill_between(test_ps, means - 1.96*sems, means + 1.96*sems, alpha=0.3)
    
    if 'success_rate' in metric:
        ax.axhline(50, color='red', linestyle='--', linewidth=2, label='50% Threshold')
    
    ax.set_xlabel('Test Failure Probability (%)')
    ax.set_ylabel(metric.replace('_', ' ').title())
    ax.set_title(f'Experiment 2: {metric.replace("_", " ").title()}')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(output_dir / f'exp2_{metric}.png', dpi=300, bbox_inches='tight')
    plt.close()


def create_all_exp1_plots(df, output_dir=Path("results/exp1/plots")):
    print("\nGenerating Experiment 1 plots...")
    plot_exp1_heatmap(df, 'success_rate', output_dir)
    plot_exp1_heatmap(df, 'mean_return', output_dir)
    plot_exp1_heatmap(df, 'crash_rate', output_dir)
    plot_exp1_robustness_curves(df, output_dir)
    print("✓ All Experiment 1 plots generated")


def create_all_exp2_plots(df, output_dir=Path("results/exp2/plots")):
    print("\nGenerating Experiment 2 plots...")
    plot_exp2_performance(df, 'success_rate', output_dir)
    plot_exp2_performance(df, 'crash_rate', output_dir)
    plot_exp2_performance(df, 'mean_time_to_failure', output_dir)
    plot_exp2_performance(df, 'mean_pre_fault_return', output_dir)
    plot_exp2_performance(df, 'mean_post_fault_return', output_dir)
    print("✓ All Experiment 2 plots generated")

print("✓ Visualization module loaded")

✓ Visualization module loaded


### Analysis

In [21]:
def analyze_exp1(df):
    print("\n" + "="*70)
    print("EXPERIMENT 1: Analysis")
    print("="*70)
    
    print("\nBest training probability for each test level:")
    for test_p in FAILURE_PROBS:
        best_train = df[df['test_p'] == test_p].groupby('train_p')['success_rate'].mean().idxmax()
        best_rate = df[(df['test_p'] == test_p) & (df['train_p'] == best_train)]['success_rate'].mean()
        print(f"  Test p={test_p:.2f}: Best train p={best_train:.2f} (success={best_rate*100:.1f}%)")
    
    on_dist = df[df['train_p'] == df['test_p']]['success_rate'].mean()
    off_dist = df[df['train_p'] != df['test_p']]['success_rate'].mean()
    print(f"\nGeneralization gap:")
    print(f"  On-distribution:  {on_dist*100:.1f}%")
    print(f"  Off-distribution: {off_dist*100:.1f}%")
    print(f"  Gap: {(on_dist - off_dist)*100:.1f} pp")


def analyze_exp2(df):
    print("\n" + "="*70)
    print("EXPERIMENT 2: Analysis")
    print("="*70)
    
    grouped = df.groupby('test_p')['success_rate'].mean()
    critical_p = None
    for p, rate in grouped.items():
        if rate < 0.5:
            critical_p = p
            break
    
    if critical_p:
        print(f"\nCritical failure threshold: p={critical_p*100:.0f}%")
    else:
        print("\nNo critical failure threshold (robust up to 25%)")
    
    p0 = df[df['test_p'] == 0.0]['success_rate'].mean()
    p25 = df[df['test_p'] == 0.25]['success_rate'].mean()
    print(f"\nPerformance degradation:")
    print(f"  p=0%:  {p0*100:.1f}%")
    print(f"  p=25%: {p25*100:.1f}%")
    print(f"  Drop:  {(p0 - p25)*100:.1f} pp")
    
    print(f"\nTime-to-failure analysis:")
    for p in FAILURE_PROBS:
        ttf = df[df['test_p'] == p]['mean_time_to_failure'].mean()
        print(f"  p={p:.2f}: {ttf:.1f} steps")


def compare_experiments(df1, df2):
    print("\n" + "="*70)
    print("COMPARATIVE ANALYSIS")
    print("="*70)
    
    print("\nZero-shot vs Matched training:")
    for test_p in FAILURE_PROBS:
        exp2_rate = df2[df2['test_p'] == test_p]['success_rate'].mean()
        exp1_rate = df1[(df1['train_p'] == test_p) & (df1['test_p'] == test_p)]['success_rate'].mean()
        improvement = (exp1_rate - exp2_rate) * 100
        print(f"  p={test_p:.2f}: Zero-shot={exp2_rate*100:5.1f}%, "
              f"Matched={exp1_rate*100:5.1f}%, Δ={improvement:+5.1f}pp")

print("✓ Analysis module loaded")

✓ Analysis module loaded


### Main Execution

In [22]:
def run_complete_pipeline():
    """Run complete research pipeline"""
    
    RUN_TRAINING_EXP1 = False
    RUN_TRAINING_EXP2 = False
    RUN_EVALUATION_EXP1 = True
    RUN_EVALUATION_EXP2 = True
    GENERATE_PLOTS = True
    RUN_ANALYSIS = True
    
    if RUN_TRAINING_EXP1:
        models_exp1 = train_experiment1(Path("models/exp1"), verbose=1)
    
    if RUN_EVALUATION_EXP1:
        models_exp1 = load_all_models_exp1(Path("models/exp1"))
        df_exp1 = evaluate_experiment1(models_exp1, N_EVAL_EPISODES, Path("results/exp1"))
        if GENERATE_PLOTS:
            create_all_exp1_plots(df_exp1, Path("results/exp1/plots"))
        if RUN_ANALYSIS:
            analyze_exp1(df_exp1)
    
    if RUN_TRAINING_EXP2:
        models_exp2 = train_experiment2(Path("models/exp2"), verbose=1)
    
    if RUN_EVALUATION_EXP2:
        models_exp2 = load_all_models_exp2(Path("models/exp2"))
        df_exp2 = evaluate_experiment2(models_exp2, N_EVAL_EPISODES, 
                                       record_gifs=True, output_dir=Path("results/exp2"))
        if GENERATE_PLOTS:
            create_all_exp2_plots(df_exp2, Path("results/exp2/plots"))
        if RUN_ANALYSIS:
            analyze_exp2(df_exp2)
    
    if RUN_ANALYSIS and RUN_EVALUATION_EXP1 and RUN_EVALUATION_EXP2:
        compare_experiments(df_exp1, df_exp2)
    
    print("\n" + "="*70)
    print("✓ PIPELINE COMPLETE")
    print("="*70)


def quick_test():
    """Quick test of corrected environment"""
    print("\n" + "="*70)
    print("TESTING CORRECTED ENVIRONMENT")
    print("="*70)
    
    for p in [0.0, 0.15, 0.25]:
        print(f"\nTesting p={p:.2f}:")
        env = make_stochastic_lunar_lander(failure_prob=p, seed=42)
        obs, info = env.reset()
        print(f"  Active family: {info.get('active_family')}")
        print(f"  Failure occurred: {env.failure_occurred}")
        
        for step in range(50):
            action = env.action_space.sample()
            obs, reward, terminated, truncated, info = env.step(action)
            if env.failure_occurred and env.failure_step == step:
                print(f"  First failure at step {step}")
            if terminated or truncated:
                break
        
        env.close()
        print(f"  ✓ Episode complete (length={step+1})")
    
    print("\n✓ All tests passed!")


# Jupyter helpers
def jupyter_train_exp1():
    return train_experiment1(Path("models/exp1"), verbose=1)

def jupyter_train_exp2():
    return train_experiment2(Path("models/exp2"), verbose=1)

def jupyter_evaluate_exp1():
    models = load_all_models_exp1(Path("models/exp1"))
    df = evaluate_experiment1(models, N_EVAL_EPISODES, Path("results/exp1"))
    create_all_exp1_plots(df, Path("results/exp1/plots"))
    analyze_exp1(df)
    return df

def jupyter_evaluate_exp2(record_gifs=True):
    models = load_all_models_exp2(Path("models/exp2"))
    df = evaluate_experiment2(models, N_EVAL_EPISODES, record_gifs, Path("results/exp2"))
    create_all_exp2_plots(df, Path("results/exp2/plots"))
    analyze_exp2(df)
    return df


print("\n" + "="*70)
print("✓ ALL MODULES LOADED - CORRECTIONS APPLIED")
print("="*70)



✓ ALL MODULES LOADED - CORRECTIONS APPLIED


The cell below has already been executed and the results were saved!!!!!

In [ ]:
if __name__ == "__main__":
    print("\n" + "="*70)
    print("STARTING AUTOMATED EXECUTION")
    print("="*70)
    print("\nWhat would you like to run?")
    print("1. Quick test (verify corrections work)")
    print("2. Train Experiment 1 (6-10 hours, 30 agents)")
    print("3. Train Experiment 2 (2-3 hours, 5 agents)")
    print("4. Evaluate Experiment 1 (requires trained models)")
    print("5. Evaluate Experiment 2 (requires trained models)")
    print("6. Full analysis (requires results CSVs)")
    print("7. Complete pipeline (train + evaluate everything)")
    
    # AUTO-RUN: Quick test by default to show it works
    print("\n" + "="*70)
    print("AUTO-RUNNING: QUICK TEST")
    print("="*70)
    quick_test()
    
    print("\n" + "="*70)
    print("QUICK TEST COMPLETE!")
    print("="*70)
    print("\nTo run full experiments, uncomment the section below:")
    print("Or use these commands in a new cell:")
    print("  jupyter_train_exp1()      # Train experiment 1")
    print("  jupyter_train_exp2()      # Train experiment 2")
    print("  jupyter_evaluate_exp1()   # Evaluate experiment 1")
    print("  jupyter_evaluate_exp2()   # Evaluate experiment 2")
    print("  run_complete_pipeline()   # Run everything")
    
    
    print("\n" + "="*70)
    print("RUNNING COMPLETE PIPELINE")
    print("="*70)
    
    # Experiment 1: Train 30 agents
    print("\n### EXPERIMENT 1: TRAINING ###")
    models_exp1 = train_experiment1(Path("models/exp1"), verbose=1)
    
    # Experiment 1: Evaluate
    print("\n### EXPERIMENT 1: EVALUATION ###")
    df_exp1 = evaluate_experiment1(models_exp1, N_EVAL_EPISODES, Path("results/exp1"))
    create_all_exp1_plots(df_exp1, Path("results/exp1/plots"))
    analyze_exp1(df_exp1)
    
    # Experiment 2: Train 5 agents
    print("\n### EXPERIMENT 2: TRAINING ###")
    models_exp2 = train_experiment2(Path("models/exp2"), verbose=1)
    
    # Experiment 2: Evaluate
    print("\n### EXPERIMENT 2: EVALUATION ###")
    df_exp2 = evaluate_experiment2(models_exp2, N_EVAL_EPISODES, 
                                   record_gifs=True, output_dir=Path("results/exp2"))
    create_all_exp2_plots(df_exp2, Path("results/exp2/plots"))
    analyze_exp2(df_exp2)
    
    # Comparative analysis
    print("\n### COMPARATIVE ANALYSIS ###")
    compare_experiments(df_exp1, df_exp2)
    
    print("\n" + "="*70)
    print("✓✓✓ COMPLETE PIPELINE FINISHED ✓✓✓")
    print("="*70)
    print("\nResults saved to:")
    print("  models/exp1/          - 30 trained agents")
    print("  models/exp2/          - 5 trained agents")
    print("  results/exp1/         - Cross-evaluation results")
    print("  results/exp1/plots/   - Heatmaps and curves")
    print("  results/exp2/         - Zero-shot results")
    print("  results/exp2/plots/   - Performance plots")
    print("  results/exp2/gifs/    - Evaluation videos")
    

Fixing the error above using the results

In [23]:
# FIX: Create directories BEFORE saving plots, then continue analysis
print("\n" + "="*70)
print("RESUMING FROM PLOT GENERATION (11 hours of work preserved)")
print("="*70)

# Create missing directories
Path("results/exp2/plots").mkdir(parents=True, exist_ok=True)
Path("results/exp1/plots").mkdir(parents=True, exist_ok=True)

# Regenerate the plots that failed
print("\n### REGENERATING PLOTS (Exp2) ###")
create_all_exp2_plots(df_exp2, Path("results/exp2/plots"))

print("\n### RUNNING ANALYSIS ###")
analyze_exp2(df_exp2)

# Comparative analysis
print("\n### COMPARATIVE ANALYSIS ###")
compare_experiments(df_exp1, df_exp2)

print("\n" + "="*70)
print("✓✓✓ COMPLETE PIPELINE FINISHED ✓✓✓")
print("="*70)
print("\nResults saved to:")
print("  models/exp1/          - 30 trained agents")
print("  models/exp2/          - 5 trained agents")
print("  results/exp1/         - Cross-evaluation results")
print("  results/exp1/plots/   - Heatmaps and curves")
print("  results/exp2/         - Zero-shot results")
print("  results/exp2/plots/   - Performance plots ✓ FIXED")
print("  results/exp2/gifs/    - Evaluation videos")


RESUMING FROM PLOT GENERATION (11 hours of work preserved)

### REGENERATING PLOTS (Exp2) ###

Generating Experiment 2 plots...
✓ All Experiment 2 plots generated

### RUNNING ANALYSIS ###

EXPERIMENT 2: Analysis

Critical failure threshold: p=10%

Performance degradation:
  p=0%:  51.0%
  p=25%: 44.0%
  Drop:  7.0 pp

Time-to-failure analysis:
  p=0.00: 588.5 steps
  p=0.05: 587.7 steps
  p=0.10: 568.7 steps
  p=0.15: 525.1 steps
  p=0.20: 487.3 steps
  p=0.25: 486.7 steps

### COMPARATIVE ANALYSIS ###

COMPARATIVE ANALYSIS

Zero-shot vs Matched training:
  p=0.00: Zero-shot= 51.0%, Matched= 51.0%, Δ= +0.0pp
  p=0.05: Zero-shot= 50.0%, Matched= 33.6%, Δ=-16.4pp
  p=0.10: Zero-shot= 49.0%, Matched= 32.6%, Δ=-16.4pp
  p=0.15: Zero-shot= 47.2%, Matched= 39.2%, Δ= -8.0pp
  p=0.20: Zero-shot= 47.0%, Matched= 33.4%, Δ=-13.6pp
  p=0.25: Zero-shot= 44.0%, Matched= 31.8%, Δ=-12.2pp

✓✓✓ COMPLETE PIPELINE FINISHED ✓✓✓

Results saved to:
  models/exp1/          - 30 trained agents
  models/exp2